# 本文主要记录YOLO v2的实现过程

---

## 实现思路
1. 依据数据格式，实现“数据读取”功能；
2. 基础主干网络ResNet-18实现；
3. 实现head，形成YOLO v2整体模型；
4. YOLO v2 损失函数实现：网络输出解码、label编码、计算损失；
5. 边边角角：配置与训练脚本、测试脚本、预测脚本，等等；
6. 进阶修改：损失函数修改，主干网络修改，等等。

---

## 本项目中需注意的功能

1. 数据预处理与标签的协同：由于图片在预处理阶段，会进行维持原图比例进行resize的操作，而标签坐标由于是相对原图的比例，故resize后的标签坐标也要进行变换；
1. `tf.keras.model.fit`函数中日志打印部分，在总损失的基础上，增加各项损失函数的输出；
1. 每个输入图片对应的目标label不定长，需pad到等长才能使用`tf.data.Dataset`对象构造训练数据集；
1. 本项目的YOLO v2的实现，相比其他TensorFlow版本的实现，**更加接近原论文、Darknet、Caffe版本的实现**；
1. 新增单一类别检测时，可以设置只有坐标回归项和IOU项，不包含类别损失项；
1. 多尺度训练：输入不同尺度的图片进行训练。


### 数据预处理与标签的协同

在图片进行原比例resize的同时，对label坐标进行等效变换。

### 日志打印

本项目使用`tf.keras.model.fit`函数进行训练，日志打印调用的是`keras.callbacks.ProgbarLogger`。由于该类只打印`tf.keras.model`对象里损失函数所返回的总loss，而我还想查看各部分loss的情况，所以可以自定义一个继承该类的类，并放置到回调函数列表中，在调用`tf.keras.model.fit`函数时传入即可。

实现过程中主要考虑的点有三个：
1. 需要损失函数的细节部分：**前期预矫正所有grid、所有anchor的xywh的`rectified_coord_loss`，中心点偏移损失`coord_loss_xy`，长宽偏差损失`coord_loss_wh`，背景anchors的IOU损失`noobj_iou_loss`，前景anchors的IOU损失`obj_iou_loss`，分类损失`class_loss`，正则项损失`regularization_loss`。**
2. 更新这些细节损失的值：在自定义损失函数的`__init__`函数里，定义了以上各项损失的TensorFlow变量，然后在每次计算总损失时，更新各项损失的变量的值；
3. 在日志打印类里，增加这些变量到打印列表中：在继承了`keras.callbacks.ProgbarLogger`类——即`utils/logger_callback.py`中`DetailLossProgbarLogger`类——后，在继承的`on_epoch_end()`函数中，给`self.log_values`字典添加以上这几项损失。之所以只重载`on_epoch_end()`函数，是因为我在调用`tf.keras.model.fit`函数时，传参`verbose=2`，此时日志只会打印一个epoch结束时的`self.log_values`；如果传入的参数`verbose=1`，此时日志会在每个batch结束时打印`self.log_values`，故继承时需要重载`on_batch_end()`函数。

以上三点，可以在源码中找到：`yolov2/trainer.py`的`train()`函数，`yolov2/yolov2_loss.py`中的`__init__()`函数和`loss()`函数，`utils/logger_callback.py`。

### 构造等长label的`tf.data.Dataset`对象


本来是可以分别构造输入图形数据集和标签数据集，然后使用组合拼装，最后拉batch和缓存，实现输入数据集的构造。

``` python
image_set = dataset.map(functools.partial(FileUtil._parse_image, image_size=image_size),
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
labels_set = dataset.map(FileUtil._parse_labels,
                 num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = tf.data.Dataset.zip((image_set, labels_set))
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
```

但是，由于同一批次里的标签数据集中每一个sample的标签pad到等长，所以需要改为：先各自构造batch，然后在拼装，最后缓存，实现输入数据集的构造。

``` python
image_set = image_set.batch(batch_size)
labels_set = labels_set.padded_batch(batch_size, (tf.TensorShape([None])), padding_values=-1.)
dataset = tf.data.Dataset.zip((image_set, labels_set))
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
```

### 遵从原版YOLO v2的实现

因为一开始使用YOLO v2使用的是Caffe版本[quhezheng/caffe_yolo_v2](https://github.com/quhezheng/caffe_yolo_v2)，看了部分源码。然后后续为了使用上的灵活，换成TensorFlow版本的。但是看了好多其他人YOLO v2、v3的比较热门的开源库后发现，其中多多少少都有点和原版对不上，包括：

1. **什么才是response_anchor?**

原实现中指出，目标中心点所在的grid中，所有的预定义anchors的预测bounding box与目标bounding box的IOU最大的anchor，为response_anchor！

这里，一般其他TensorFlow库存在问题：并不是在损失计算阶段，使用预测bounding box与目标bounding box计算IOU；而是在label数据预处理阶段，使用anchor与目标bounding box计算IOU。这对于长宽比较靠近某个预定义anchor的目标可能问题不大，但是对于那些长宽在多个预定anchor边缘的目标而言，可能这种硬切分最终不利于训练。

第二个问题是，最大的IOU。YOLO v3论文中，作者介绍了一些try but don't work的tricks，包括：设定两个阈值A<B,$IOU\in[0, A]$计算背景损失，$IOU\in[A, B]$不计算损失，$IOU\in[B, 1]$计算前景损失，我发现有些库用了这种做法。

2. **损失函数用MSE还是CE？**

因为损失函数的公式，只有YOLO v1的论文写的均方差（MSE）损失，后续的论文没写，而源码中使用的也是MSE损失。那么在我们自己实际训练过程中，对于**分类损失和Score损失，究竟使用均方差（MSE）损失好还是交叉熵（CE）损失呢？**

我在`1_learning_note/Use_Cross_Entropy_or_Mean_Square_Error_Loss.ipynb`中，对比了两者，发现：
1. MSE损失虽然对预测误差是二次关系，但CE损失对预测误差是$[0,1]$间的对数增长关系，更加敏感（也就是loss会更加大）；
2. MSE损失存在在梯度 小-大-小 的波动现象，而CE损失函数的梯度和误差是单调增关系，更有利于对大误差样本的学习。

基于上面对CE损失函数和MSE损失函数的认识，我们先做数值的直观感受：
1. 在与目标值误差为0.1时，MSE损失函数值为0.01，CE损失函数值为log(0.9)=0.046;
2. 在与目标值误差为0.3时，MSE损失函数值为0.09，CE损失函数值为log(0.7)=0.155;

所以如果我们以$[0.1,0.3]$为允许的误差范围，参考原论文各项损失的权重为：前期MSE校正坐标损失权重系数为0.01，MSE坐标损失权重系数为5，MSE背景Score损失权重系数为0.5，MSE前景Score损失权重系数为1，MSE类别损失权重系数为1。故将分类损失和Score损失从MSE损失改为CE损失时，是否可以把除坐标损失外的其他损失项的权重系数，用$[1.5, 5]$范围的除数来调参协调各项损失。当然，别忘了，这里有L2正则化损失项的权重系数。不过感觉可以将YOLOv2损失权重系数和L2正则化损失权重系数作为两个体系大概调整。

在`yolov2/yolov2_loss.py`的`loss()`函数里，分类损失和Score损失实现的是CE损失函数，但是也加了MSE损失的注释。当然，可能最好的方式是：基于理论理解上的实践吧。



最后，`yolov2/yolov2_loss.py`里，**损失函数计算的主要步骤**为：
1. 利用`tf.map_fn`对每个sample逐一计算损失；
2. 计算所有grid、anchor的预测bounding box和实际目标bounding box的最大IOU，作为后续判断该anchor是否需要计算背景损失的依据；
3. 取出实际目标中心所在grid的anchor，计算与对应的目标的IOU，得到最大IOU的anchor的位置，作为后续计算前景损失的依据。

### 单类别检测可以设置不包含类别损失项

实时上，由于YOLO系列算法的前背景是通过IOU来区分的，所以当只有单类别的检测时，是可以舍弃掉类别损失项的。于是，我用`config.py`中包含的`FLAGS.class_num`来标记类别数外，还用于标记是否要包含类别损失项：`FLAGS.class_num=0`表示单类别但不包含类别损失项。此时，YOLO v2网络的输出不包含类别的channel，在做网络输出解码、标签转换、损失计算时，都相应的不会计算类别损失！

### 多尺度训练

**聚类anchor的时候，需要计算归一化的anchor，这样可以使anchor和实际图形尺度解耦。**

由于训练过程中数据预处理的原因，可能使得实际图像尺度变化，如果此时再用这批数据去聚类anchor：
1. 如果是归一化的操作，聚类出来的anchor与预处理前聚类出来的anchor是一致的；
2. 而如果不是归一化的anchor，那么聚类出来的anchor尺度和最后的feature map大小（如$13 \times 13$）是相关的（例如是归一化的anchor*feature map尺寸）;而由于输入尺度的变化会造成输出尺度变化，最终导致聚类得到的预定义anchor和实际训练过程中数据潜在的anchor不一致！

除了多尺度训练时，使用归一化的anchor之外，训练时最好也不要用crop这些改变目标相比原图比例的技巧。

---

## 预告

我也出了YOLO v3的代码库，欢迎start~

同时，后续想出R-CNN系列的代码库，基于别人开源库基础上改改~